<a href="https://www.kaggle.com/alperenkaran/spotify-top50-charts-analysis?scriptVersionId=88886338" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<h1><center>Spotify Top50 Chart Analysis</center></h1>

<h1><center><img src="https://developer.spotify.com/assets/branding-guidelines/logo@2x.png" alt="Spotify" style="width: 800px;"/></center></h1>

In [ ]:
import numpy as np
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def visualize_word_counts(counts):
    wc = WordCloud(max_font_size=130, min_font_size=25, colormap='tab20', background_color='white', 
                   prefer_horizontal=.95, width=2100, height=700, random_state=0)
    cloud = wc.generate_from_frequencies(counts)
    plt.figure(figsize=(18,15))
    plt.imshow(cloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

# Read the csv file, and get the Top50 lists for each day

In [ ]:
usecols = ['title','rank','date','artist','region','chart','streams']  # columns that we will read from the csv
charts = pd.read_csv('../input/spotify-charts/charts.csv', usecols=usecols)  # read the csv file
top50 = charts[(charts['chart'] == 'top200') & (charts['rank'] <= 50)]  # get the top50
top50 = top50.drop(['chart'], axis=1).reset_index(drop=True)  # we don't need the chart column anymore
top50

# A visualization of artist popularity

WordCloud is a good option to visualize artist popularity. 

We first create a dictionary `counts` which counts how many times an artist appear in the `top50` dataframe. 

Then, we visualize this dictionary using WordCloud.

In [ ]:
counts = top50['artist'].value_counts()
visualize_word_counts(counts)

# How many days does a song stay in Global Top50 ?

Some songs appear in the Global Top50 for a few days, but some hit songs are there like ... forever.

For example, I remember Ed Sheeran's **Shape of You** was always in the Global Top50 list.

In [ ]:
# we create a dataframe called n_days with columns: 'artist', 'title', 'Number of days in Global Top50'
top50_global = top50[top50['region'] == 'Global']
n_days = top50_global.groupby(['artist', 'title'])['date'].count().reset_index()
n_days.columns = ['artist', 'title', 'Number of days in Global Top50']

In [ ]:
plt.figure(figsize=(15,4))
plt.hist(n_days['Number of days in Global Top50'], bins=50, color ='blueviolet', edgecolor='black', linewidth=1.2)
plt.yscale('log')
plt.title('How many days will a song remain on Global Top50 ?')
plt.xlabel('Number of days in Global Top50')
plt.ylabel('Number of songs')
plt.locator_params(axis='x', nbins=25)  # more ticks on x-axis
plt.show()

As we expected most songs were there just about one or two months (notice the logscale on the y-axis).

There are some songs which were on Global Top50 for over **two years**! Which songs were they? Can you guess?

In [ ]:
n_days.sort_values(by='Number of days in Global Top50', ascending=False)

# Streams by region

Which region's Top50 list is streamed the most? The result will be somewhat related to the region's population, but not directly. 

For example, it is likely that US Top50 list is streamed by listeners all over the world, not only by US citizens.

In [ ]:
# compute total streams by region
streams = top50.groupby('region')['streams'].sum().reset_index()

# compute percent stream
streams['percent_streams'] = streams['streams']/streams['streams'].sum()

# rename regions with very little streams (< .01 %) as 'Other'
streams['region'] = streams.apply(lambda x: x['region'] if x['percent_streams'] >= .01 else 'Other', axis=1)

# we need another groupby because there are multiple regions with name 'Other'
streams = streams.groupby('region')['percent_streams'].sum().reset_index().round(3).sort_values(by='percent_streams')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.pie(x=streams['percent_streams'], labels=streams['region'], autopct='%.1f%%')
ax.set_title('Streams by region')
plt.tight_layout()

# Is rank related to number of streams?

We expect that whenever rank is small number (like 1st rank) the number of streams will be high. So, they must be inversely correlated.

Let's look at it day by day in the Global Top50.

In [ ]:
correlations = [df['streams'].corr(df['rank']) for date, df in top50_global.groupby('date')]

plt.figure(figsize=(15,4))
plt.hist(correlations, bins=50, color ='teal', edgecolor='black', linewidth=1.2)
plt.xlabel('Correlation')
plt.ylabel('Number of days')
plt.title('Correlation of rank with streams')
plt.show()

The histogram shows that on most days, the rank and the number of streams are highly negatively correlated. For only a few days, they were moderately negatively correlated.

# Does the rank of a song change over time?

Consider the song **Blinding Lights** which stayed in Global Top50 for the longest time. We wonder how the rank of the song changed over time.

In [ ]:
blinding_lights = top50_global[top50_global['title'] == 'Blinding Lights'].sort_values(by='date').reset_index(drop=True)
blinding_lights

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(blinding_lights['rank'], color='maroon')
plt.show()

At about 450 days, the song's rank started dropping.

But this plot looks kinda messy, doesn't it? Perhaps we could look at moving monthly averages for rank, instead of daily rankings.

Computing a moving mean is very easy with pandas, just use the **rolling** method!

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(blinding_lights['rank'].rolling(30).mean(), color='maroon')
plt.show()